# Talktorial 12

# Binding Site Detection (DogSiteScorer)

Abishek Laxmanan Ravi Shankar, supervised by Dominique Sydow and Jaime Rodriguez Guerra

## Aim of the talktorial

In this talktorial, we **detect the ligand binding site** of a protein structure at the example of **EGFR** using **DoGSiteScorer** protein.plus web server and later **validate the results** with the binding pocket detected using **KLIFS API**. Then comparing the percentage of coverage of DoGSiteScorer in accodance with the KLIFS results. 

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Fig1_Work_flow.jpg" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 1: Workflow of the Talktorial - Detection of Binding site with DoGSiteScorer and validation using KLIFS (Figure created by Abishek Laxmanan Ravi Shankar and Dominique Sydow)
</div>

# Learning goals

### Theory
* Protein binding sites
* Binding site detection 
    * Introduction
        * Geometry-based approaches
        * Energy-based approaches
    * DogSiteScorer    
        * pocket detection
        * Druggability
* Validation
    * KLIFS pocket definitions

### Practical

* Binding site detection using DoGSiteScorer
    * Submit job to DoGSiteScorer and get job location
    * Get DoGSiteScorer pocket metadata
    * Sort pockets based on their drugScore
    * Get best binding site file content
    * Get residue IDs and names of best pocket
* Validation using KLIFS
    * Get KLIFS structure ID for a PDB ID of interest
    * Get pocket from KLIFS ID and extract residues
* Comparison DoGSiteScorer and KLIFS pocket
    * Get DoGSiteScorer coverage with KLIFS results



## References

* Combining global and local measures for structure-based druggability predictions.
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

* Prediction, Analysis, and Comparison of Active Sites 
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

* DoGSiteScorer
([<i>Bioinformatics</i> 2012, <b>28</b>, 2074-2075](https://doi.org/10.1093/bioinformatics/bts310)) 

* sc-PDB:  an Annotated Database of Druggable Binding Sites from the Protein Data Bank
([<i>Journal of Chemical Information and Modeling</i> 2006, <b>46</b>, 717-727](https://pubs.acs.org/doi/abs/10.1021/ci050372x))

* Fpocket
([<i>BMC Bioinformatics</i> 2009, <b>10</b>, 168](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168))

* Protein–ligand complex
([Protein complex wiki](https://en.wikipedia.org/wiki/Protein%E2%80%93ligand_complex))

* Voronoi tessellation
([Voronoi tessellation Wiki](https://en.wikipedia.org/wiki/Voronoi_diagram))

* KLIFS database
( [KLIFS Website](http://klifs.vu-compmedchem.nl/))  

* Fragments based drug Discovery
([Google books](https://books.google.de/books?id=tjfvCQAAQBAJ&pg=PA202&lpg=PA202&dq=Fragments+based+drug+Discovery+SURFNET&source=bl&ots=O6XVGMVPTV&sig=ACfU3U2K6btUeMpgN4FBe627jpIJL6-mtQ&hl=en&sa=X&ved=2ahUKEwjonezYmbLkAhXBRxUIHdzKClIQ6AEwAnoECAkQAQ))

* KLIFS: a structural kinase-ligand interaction database
([<i>KLIFS Pubmed</i> 2016, Nucleic Acids Research, <b>44</b>, 365-371](https://www.ncbi.nlm.nih.gov/pubmed/26496949)) 

* KLIFS: A Knowledge-Based Structural Database To Navigate Kinase−Ligand Interaction Space
([<i>KLIFS Journal of Medicinal Chemistry</i> 2013, <b>57</b>, 249-257](https://pubs.acs.org/doi/10.1021/jm400378w))

* SURFNET
([<i>Journal of Molecular Graphics</i>1995, <b>13</b>323-330](https://www.ncbi.nlm.nih.gov/pubmed/8603061))

* LIGSITE
([<i>Journal of Molecular Graphics and Modelling</i> 1997 <b>6</b> 359-363](https://www.sciencedirect.com/science/article/pii/S1093326398000023?via%3Dihub))

* Docking 
( [Docking wiki](https://en.wikipedia.org/wiki/Docking_(molecular)))

* DrugSite
([<i>Genome Informatics</i>2004 <b>15</b> 31–41 ](https://pdfs.semanticscholar.org/59ca/aac9a79c29660385af0ab83b82fba57cc476.pdf))

* Metapocket 2.0
([Summary of various binding site detection approaches](https://projects.biotec.tu-dresden.de/metapocket/algorithm.php))



## Theory

### Protein binding sites

The main objective of a pharmaceutical research is to find a new drug inorder to cure a specific disease. Structure-based approaches in drug design develop drugs with respect to a target protein which is associated with the disease under investigation. However, in order to develop such a drug it is crucial to know where the drug is supposed to bind in the protein and therefore inhibit or activate the protein. Not always such a small molecule binding site is known. Therefore it is important to have tools for binding site detection.

Binding site detection has a special mention in the process of drug development. Binding sites are defined as a **hollow 3 dimensional space** on the surface of a protein molecule, which serves as the region for **ligand**, **peptide**, or **protein** binding. There are various factors which influence whether such a region is suitable binding site.
* The Shape
* The Spatial arrangement of the residues
* The Possible interactions are
    * Van der Waals interactions
    * Hydrogen bond interactions
    * Hydrophobic regions
    * Electrostatic interactions
    * π-π interactions
   


### Binding site detection 

### Introduction 

If ligand information is available (protein-ligand-complex), then the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).
If the ligand is absent, then the detection tools can be used in the process of pocket detection. This comprises of 2 standard approaches. These methods can be grouped into, **geometry-based and energy-based** approaches as well as **grid-based and grid-free** approaches as outlined in the Figure 1.

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Figure2_DetectionMethods.png" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 2: Binding site detection methods can be grouped into geometry-based and energy-based approaches as well as grid-based and grid-free approaches.
    
Figure from Prediction, Analysis, and Comparison of Active Sites. Volkamer et al., 2018
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))
</div>

#### Geometry-based approaches

Geometry-based approaches analyze the shape of a molecular surface to locate cavities. It is based upon the 3D spatial arrangement of the atoms on the protein surface. They are broadly divided into two namely Grid-Based and Grid-free approaches.

* **Grid-based approaches**

    * Examples: POCKET, LIGSITE and DoGSiteScorer 
    * In LIGSITE, the cartesian coordinates runs along the X, Y and Z axes through a 3D region which consists of protein surface on both sides.
    * In addition to these axes a cubic diagonal passes along to find out the width of the cleft in between the protein surface. Therefore LIGSITE checks in seven directions the distance of each atom in the protein surface.
    * Therefore these coordinates enables LIGSITE to diffrentate the Protein Solvent Protein regions (PSP) on the protein surface
    * DoGSiteScorer is explained in detail below.
   
* **Grid-free approaches**

    * Example: SURFNET
    * In SURFNET spheres are placed on the protein surface instead of having grids. 
    * The probe spheres are placed midway between the pairs of atoms on the protein. Incase a probe overlaps any nearby atom, its radius is reduced until no overlap occurs. 
    * The resulting probes defines the pocket and the cavities. The cutoff radius of the spheres are of 1 to 4 angstrom .

#### Energy-based approaches

Interaction of a molecular fragment with protein is recorded. Favourable energitic responses are assigned to pockets. DrugSite and Docking are the well known energy-based methods. They are again divided into grid-based and grid-free approaches.
<div class="center" align="right" width="500"> </div>

* **Grid-based approaches**

    * Example: Drugsite
    * This method uses carbon probes placed on each grid point, and van der Wall's energies between the probe and the protein with distance less than 8 Å are calculated. Mean energy and Standard deviation are calculated for these grid points. 
    * Grid points with unfavourable energies are removed. Grid points fulfilling these cut-off are merged to pockets.
   
* **Grid-free approaches**

    * Example: Docking
    * Docking process is similar to "lock and key" model in which the **target protein resembles** the **lock** and **ligand** to the **key**
    * In this method, a scoring function is incorporated to evaluate the fragments that are placed at a position on the protein surface.
    * Scoring functions are physics based, which uses Molecular Mechanics force fields, which primarily estimates the amount of affinity between the binding ligand and the protein.
    * The scoring functions calculates the feasibilty of whether the ligand could be docked to the particular protein
    * These pockets are assigned based on the quantity of fragments that bind to a specific area.

### DogSiteScorer 

Certain geometric and physiochemical properties like pocket depth, pocket volume and surface are calculated in an automatic manner for the predicted pockets and subpockets. In **DogSiteScorer** , pockets and subpockets are predicted with DoGSite using a Difference of **Gaussian filter**. DogSiteScorer falls under the category of Geometry-based and Grid-based approaches. Pocket volume and surface are calculated by counting the **grid points** constituting the pocket volume or its surface and multiplying this number with the grid box volume or surface, respectively. A **breadth-first search** algorithm is used for pocket depth computation, starting from the solvent exposed pocket parts toward the most deeply buried regions. 

#### Pocket Detection

* The protein structure is covered around with a grid of coordinates.
* Each of the **grid points** are surrounded with 26 grid points. Each inner grip point comprises of 26 grid points as neighbours. The grid points at the border have 25 grid points as its neighbouring grids. 
* The grid points at the border are termed as 'hull grid points' and if they are within a certain distance then they could be  concluded as 'protein surface'.
* Volume could be defined as a product of 'volume grid points' and 'cubed grid spacing'.
* The depth is defined as 'the maximum possible distance between the solvent and the buried part of the pocket'
* Difference of **Gaussian filter** is applied across the edges of the grid.
* This operation helps to find the position on a protein surface where the location of a **sphere-like object** is favorable.
* The last step in this algorithm is the merging of neighbouring subpockets to the pockets.

<div style="width:400; font-size:100%; text-align:center;">
    <img src="images/Figure3_BindingSitePocket.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    Figure 3: The binding site on the surface of the protein shown with grid view consisting of the surface molecules, volume inside the binding region, and the region exposed to the exterior of the protein 
    
Figure from "Combining global and local measures for structure-based druggability predictions."
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))
</div>

#### Druggability 

* The Druggability is based on Machine learning technique - **Support Vector Machine (SVM)**.
* Two strategies were used for the Druggability prediction, which includes, **Global machine learning approach** and **Nearest neighbour approach**. 
* **Discriminative Analysis** - Best suited to seperate druggable from undruggable.
* The model has been trained and tested on both the redundant and non-redundant version of the druggable dataset
* This model showed a mean accuracy of 90%.
* The druggabilty score lies between 0 and 1. The higher the score the better the chance that the pocket is druggable.
    
<div style="width:1000; font-size:100%; text-align:center;">
    <img src="images/Figure3_DogSiteAlgo.png" alt="alternate text"  width="1000" style="padding:0.8em;"/>
    Figure 4: Flowsheet explaining the pipeline of DoGSiteScorer
    
Figure from "Combining global and local measures for structure-based druggability predictions."
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

</div>


### Validation methods

Once we obtain the best binding site from the DoGSiteScorer, we can compare the results with any other method in order to validate it. Here we compare it with the KLIFS binding pocket from the KLIFS API. 

#### KLIFS pocket definition

KLIFS stands for **Kinase-Ligand Interaction Fingerprints and Structures database**. It is a structural repository of over 2900 human and mouse kinase enzymes along with their catalytic activity. It also deals with the kinase domains and also the inhibitors which can interact with them. Kinases share a similar conserved regions, which serves as a challenge for small molecules that can selectively bind to a specific kinase. 

KLIFS enables us to do systematic comparison and analysis of chemical features of all available kinases in the process of ligand binding. The classification helps us of an all-encompassing binding site of **85 residues**. It is possible to compare the **interaction patterns** of **kinase-inhibitors** to each other to, for example, identify crucial interactions determining kinase-inhibitor selectivity. KLIFS API could return the binding pocket of a specific kinase protein. This could therefore be used as an ideal method to compare the binding site with regard to DogSiteScorer for **EGFR kinase** protein. 

## Practical

In [1]:
import time

from biopandas.mol2 import PandasMol2
from biopandas.pdb import PandasPdb
import requests
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Binding site detection using DoGSiteScorer

#### Submit job to DoGSiteScorer and get job location

In [3]:
def dogsitescorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """
    Submit PDB ID to DoGSiteScorer webserver using their API and get back URL for job location.
    
    Parameters
    ----------
    pdb_id : str 
        4-letter valid PDB ID, e.g. '3w32'.
    chain : str
        Chain ID, e.g. 'A'.
    ligand : str
        Name of ligand bound to PDB structure with pdb_id, e.g. 'W32_A_1101'. 
        Currently, the ligand name must be checked manually on the DoGSiteScorer website. 
        
    Returns
    -------
    str
        Job location URL for submitted query.
        
    References
    ----------
        Function is taken from: https://github.com/volkamerlab/TeachOpenCADD/pull/3
    """
    
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    
    return r.json()['location']

In [4]:
# Identifying the job location where the work is submitted to the web server
job_location = dogsitescorer_submit_with_pdbid('3w32', 'A')
job_location

'https://proteins.plus/api/dogsite_rest/FSAdQY1ADHYFviSbwHG2jbPK'

In [5]:
# Wait a bit so that job can finish
time.sleep(30)

#### Get DoGSiteScorer pocket metadata

In [6]:
def get_dogsitescorer_metadata(job_location):
    """
    get_dogsitescorer_metadata - This function retrieves results from a DoGSiteScorer query.
    It returns the binding sites which are found over the protein surface, 
    in the form of a table with the details about all detected pockets.

    Parameters
    ----------
    job_location : str
        Consists of the location of a finished DoGSiteScorer job on the proteins.plus web server.
    
    Returns
    -------
    pandas.DataFrame
        Table with metadata on detected binding sites. 
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get URL of result table file
    result_file = result.json()['result_table']
    
    # Get result table
    result_table = requests.get(result_file).text
    
    # Split string into list of lists (=table)
    result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]

    # Remove spaces
    result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]

    # Extract column names, index names, table body
    column_names = result_table_split[0]
    index_names = [i[0] for i in result_table_split[1:]]
    table = [i[1:] for i in result_table_split[1:]]

    # Convert to DataFrame
    result_table_df = pd.DataFrame(
        table,
        columns=column_names[1:],
        index=index_names
    )
    result_table_df.index.name = 'name'
    
    # Convert number strings to numeric values
    for name, data in result_table_df.iteritems():
        try:
            result_table_df[name] = pd.to_numeric(data)
        except ValueError:
            pass
    
    return result_table_df

In [7]:
# Retrieving all the guessed pockets from DoGSiteScorer Web server 
metadata = get_dogsitescorer_metadata(job_location)
metadata

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_0_1,0.0,0.0,"""""",201.73,0.08,381.07,11.36,1.889010,-,-,0.17,0.25,51,17,9,10,0.28,0,36,6,7,2,0,0.08,0.17,0.25,0.50,1,1,0,1,1,1,0,0,0,0,3,1,1,0,0,0,1,0,0,1,0.17,0.174816
P_0_2,0.0,0.0,"""""",185.60,0.17,282.00,9.35,1.519397,-,-,0.45,0.55,48,17,8,12,0.32,0,31,8,8,1,0,0.17,0.25,0.08,0.50,0,2,0,1,0,0,1,1,0,0,2,1,1,1,0,0,0,0,0,2,0.13,0.195695
P_0_3,0.0,0.0,"""""",175.30,0.15,297.42,9.29,1.696634,-,-,0.23,0.37,48,16,8,14,0.37,0,32,8,8,0,0,0.14,0.14,0.36,0.36,1,1,1,2,0,0,0,3,0,0,1,1,0,1,1,1,0,0,0,1,0.13,0.168845
P_0_4,0.0,0.0,"""""",170.37,0.08,390.10,11.99,2.289722,-,-,0.16,0.20,47,14,7,17,0.45,0,34,7,6,0,0,0.00,0.18,0.18,0.64,2,2,0,0,0,1,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0.15,0.223742
P_0_5,0.0,0.0,"""""",90.43,0.24,177.50,6.24,1.962844,-,-,0.70,0.89,26,8,6,5,0.26,0,16,6,4,0,0,0.12,0.25,0.25,0.38,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0.00,0.165232
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489


#### Sort pockets based on their drugScore

In [8]:
# Sort the obtained binding site by descending drugScore
metadata.sort_values(by='drugScore', ascending=False)

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_3,0.0,0.0,"""""",244.16,0.04,514.94,14.34,2.109027,-,-,0.12,0.31,92,18,7,24,0.49,0,67,12,13,0,0,0.12,0.06,0.41,0.41,0,0,1,1,1,0,1,1,1,0,1,0,0,2,3,2,1,0,1,1,0.12,0.572013
P_4_1,0.0,0.0,"""""",60.93,0.20,193.96,5.80,3.183325,-,-,0.54,0.84,28,4,1,16,0.76,0,22,4,2,0,0,0.00,0.14,0.14,0.71,0,0,0,0,0,0,0,0,0,1,0,1,1,1,2,0,1,0,0,0,0.00,0.570708
P_2,0.0,0.0,"""""",286.21,0.18,462.29,11.83,1.615213,-,-,0.23,0.44,70,20,9,13,0.31,0,50,9,9,2,0,0.12,0.06,0.18,0.65,1,0,0,2,0,1,0,0,0,1,1,1,2,1,2,0,1,0,1,3,0.09,0.537137
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489
P_4,0.0,0.0,"""""",169.28,0.21,373.47,11.49,2.206226,-,-,0.12,0.16,59,14,3,24,0.59,0,44,7,7,1,0,0.08,0.08,0.17,0.67,0,0,0,1,0,1,0,0,0,3,0,1,2,1,2,0,1,0,0,0,0.07,0.424397
P_5,0.0,0.0,"""""",166.59,0.16,347.46,11.99,2.085719,-,-,0.14,0.24,58,18,8,10,0.28,0,39,8,11,0,0,0.15,0.08,0.46,0.31,0,0,1,1,0,2,1,1,1,1,1,0,1,0,1,1,0,0,1,0,0.00,0.401384


In [9]:
def select_best_pocket(metadata, by='drugScore'):
    """
    select_best_pocket - This function uses 'drugScore' as a parameter to identify the best pocket 
    among the obtained pockets.
    
    Parameters
    ----------
    metadata : pd.DataFrame
        'metadata' contains all the guessed pockets retrieved from the DoGSiteScorer website
        
    by : str
        'by' is used to sort the pockets based upon the 'drugsite' score
        
    Returns 
    -------
    str
        The best pocket based on the 'DrugScore' is returned here as the output.
    """
    
    by_methods = ['drugScore', 'volume']
    
    # Sort by best druggability score
    if by == 'drugScore':
        sorted_pocket = metadata.sort_values(by='drugScore', ascending=False)
    # Sort by volume
    elif by == 'volume':
        sorted_pocket = metadata.sort_values(by='volume', ascending=False)
    else:
        raise ValueError(f'Selection method not in list: {", ".join(by_methods)}')
                         
    # Get name of best pocket
    best_pocket_name = sorted_pocket.iloc[0, :].name     
        
    return best_pocket_name

In [10]:
# Get the name of the best pocket
best_pocket = select_best_pocket(metadata, by='drugScore')
best_pocket

'P_0'

#### Get best binding site file content

In [11]:
def get_pocket_locations(job_location):
    """
    get_pocket_locations - We have identified the best pocket by the above step. 
    Now we must obtain the best pocket file in ".pdb" format.
    
    Parameters
    ----------
    job_location : str
        This string contains the job location of the DoGSiteScorer Web server
    
    Returns
    -------
    str
        This comprises of the URLs of all the retrieved PDB ID from DoGSiteScorer Web server
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get residues
    return result.json()['residues']

In [12]:
def get_best_pocket_location(pocket_files, best_pocket):
    """
    get_best_pocket_location - To create a function to get the best binding site file from
    the list of all obtained pdb files.
    
    Parameters
    ----------
    pocket_files : list
        Contains the list of all PDB URLs retrieved from the web server
    best_pocket : str
        Stores the best binding site location from the list of all binding sites

    Returns
    ------
    str
        Returns the single URL of the best binding site (best pocket)
    """
    result = []
    
    for pocket_file in pocket_files:
        
        if f'{best_pocket}_res' in pocket_file:
            result.append(pocket_file)
            
    if len(result) > 1:
        raise TypeError(f'Multiple strings detected: {", ".join(result)}.')
    elif len(result) == 0:
        raise TypeError(f'No string detected.')
    else:
        pass
            
    return result[0]

In [13]:
# Get URL for all PDB files
pocket_files = get_pocket_locations(job_location)
pocket_files

['https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_0_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_1_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_2_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_3_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_4_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_5_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_1_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_1_0_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_1_1_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_2_res.pdb',
 'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2

In [14]:
# Get URL for PDB file containing the best pocket
best_pocket_location = get_best_pocket_location(pocket_files, best_pocket)
best_pocket_location

'https://proteins.plus/results/dogsite/FSAdQY1ADHYFviSbwHG2jbPK/3w32_P_0_res.pdb'

#### Get residue IDs and names of best pocket

In [15]:
def get_pocket_residues(pocket_location):
    """
    get_pocket_residues - Gets residue IDs and names of a pocket.
    
    Parameters
    ----------
    pocket_location : str
        This string retrieves the PDB file from the URL
        
    Returns
    -------
    Pandas dataFrame
        Returns the pandas adtaframe consisting of the list of residue names and the residue IDs 
        from the best obtained binding site
    """
    
    # Retrieve PDB file content from URL
    result = requests.get(pocket_location)

    # Get content of PDB file  
    pdb_residues = result.text
    
    # Load PDB format as DataFrame
    ppdb = PandasPdb()
    pdb_df = ppdb._construct_df(pdb_residues.splitlines(True))['ATOM']
    
    return pdb_df[['residue_number', 'residue_name']]

In [16]:
# Get residues of best pocket
pocket_residues = get_pocket_residues(best_pocket_location)
pocket_residues

,residue_number,residue_name
0,701,GLN
1,701,GLN
2,701,GLN
3,701,GLN
4,702,ALA
5,702,ALA
6,703,LEU
7,703,LEU
8,703,LEU
9,703,LEU


In [17]:
dss_residue_number = pocket_residues.residue_number
dss_residue_number

0       701
1       701
2       701
3       701
4       702
5       702
6       703
7       703
8       703
9       703
10      703
11      705
12      705
13      705
14      705
15      705
16      705
17      718
18      718
19      718
20      718
21      719
22      720
23      720
24      720
25      721
26      721
27      721
28      722
29      722
30      722
31      722
32      723
33      723
34      723
35      723
36      723
37      723
38      723
39      723
40      724
41      724
42      724
43      724
44      726
45      726
46      726
47      731
48      731
49      731
50      740
51      740
52      743
53      743
54      743
55      743
56      743
57      744
58      744
59      744
60      745
61      745
62      745
63      745
64      745
65      745
66      745
67      761
68      762
69      762
70      762
71      762
72      762
73      762
74      762
75      762
76      765
77      765
78      765
79      765
80      765
81      766
82      766
83  

# Validation using KLIFS

We have identified the binding site of a kinase of interest with the help of DoGSiteScorer web server. 
Now we want to check if the obtained binding site is in agreement with the defined binding site defined by the KLIFS database. 

In [18]:
from bravado.client import SwaggerClient

KLIFS_API_DEFINITIONS = "http://klifs.vu-compmedchem.nl/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(KLIFS_API_DEFINITIONS, config={'validate_responses': False})

#### Get KLIFS structure ID for a PDB ID of interest 

In [19]:
def get_klifsid_from_pdbid(pdb_codes):
    """
    get_klifsid_from_pdbid : Gets KLIFS structure ID for a PDB ID of interest.
    
    Parameters
    ----------
    pdb_codes : str or list of str
        The PDB ID of our desired kinase protein is given as an input
    
    Returns
    -------
    int
        The KLIFS ID value of the given PDB ID is returned
    """
    
    # If only a single PDB ID is provided in the form of a string, cast string to list
    if isinstance(pdb_codes, str):
        pdb_codes = [pdb_codes]
    
    # Get metadata for KLIFS structures associated with one or more PDB IDs
    klifs_structures = KLIFS_CLIENT.Structures.get_structures_pdb_list(
        pdb_codes=pdb_codes
    ).response().result
    
    # Get KLIFS IDs for KLIFS structures
    klifs_ids = [i.structure_ID for i in klifs_structures]
    
    # Get first KLIFS ID
    klifs_id = klifs_ids[0]
    
    return int(klifs_id)

In [20]:
# Get KLIFS ID
klifs_id = get_klifsid_from_pdbid('3w32')
klifs_id

784

#### Get pocket from KLIFS ID and extract residues

In [21]:
def get_klifs_pocket(klifs_id):
    """
    get_klifs_pocket - This function takes kLIFS ID as the input and 
    returns the binding site residues as the output
    
    Parameters
    ----------
    klifs_id : str
        An ID which is provided by the KLIFS database when the PDB ID of our desired protein is given
    
    Returns
    -------
    str
        The Mol 2 file is retrieved from the KLIFS database as the output
    """
    
    klifs_pocket = KLIFS_CLIENT.Structures.get_structure_get_pocket(
        structure_ID=klifs_id
    ).response().result
    
    return klifs_pocket

In [22]:
def get_klifs_residues(klifs_pocket):
    """
    get_klifs_residues - This function returns the residue IDs and 
    residue names which are returned from the KLIFS database 
    
    Parameters
    ----------
    klifs_pocket : str
        Creatzed to store the Mol 2 file which will be obtained from the KLIFS API
    
    Returns
    -------
    Pandas DataFrame
        Returns the residues of the binding site which is retrieved from the KlIFS database
    """
    
    ppdb = PandasMol2()
    mol2_df = ppdb._construct_df(
        klifs_pocket.splitlines(True),  
        col_names=['1','2','3','4','5','6','7','Residues','9','10'],  #TODO include reasonable column names
        col_types=[str]*10  #TODO include reasonable
    )
    
    return mol2_df[['Residues']]

In [23]:
# Get KLIFS pocket
klifs_pocket = get_klifs_pocket(klifs_id)
klifs_pocket

'@<TRIPOS>MOLECULE\nHUMAN/EGFR_3w32_chainA\n1364 1368 85 0 0 \nBIOPOLYMER\nUSER_CHARGES\n\n\n@<TRIPOS>ATOM\n   1 N        7.7187    15.4685    51.5111 N.3    1 LYS716   0.0000 BACKBONE\n   2 H        7.3300    16.1756    50.9036 H      1 LYS716   0.0000 BACKBONE\n   3 CA       7.3282    14.0740    51.3137 C.3    1 LYS716   0.0000 BACKBONE\n   4 HA       8.1219    13.4452    51.7172 H      1 LYS716   0.0000 BACKBONE\n   5 C        7.1509    13.7803    49.8284 C.2    1 LYS716   0.0000 BACKBONE\n   6 O        6.8337    14.6724    49.0417 O.2    1 LYS716   0.0000 BACKBONE\n   7 CB       6.0216    13.7553    52.0569 C.3    1 LYS716   0.0000 \n   8 HB2      5.6643    12.7873    51.7056 H      1 LYS716   0.0000 \n   9 HB3      5.2944    14.5258    51.8006 H      1 LYS716   0.0000 \n  10 CG       6.1220    13.6938    53.5737 C.3    1 LYS716   0.0000 \n  11 HG2      5.1238    13.8138    53.9949 H      1 LYS716   0.0000 \n  12 HG3      6.7606    14.5086    53.9147 H      1 LYS716   0.0000 \n  13

In [24]:
# Get KLIFS residues
klifs_residues = get_klifs_residues(klifs_pocket)
klifs_residues

,Residues
0,LYS716
1,LYS716
2,LYS716
3,LYS716
4,LYS716
5,LYS716
6,LYS716
7,LYS716
8,LYS716
9,LYS716


In [25]:
# Split residue ID and name (in order to have the same output as for the DSS pocket residues)
klifs_residues.Residues
klifs_residue_name = klifs_residues.Residues.apply(lambda x: x[:3])
klifs_residue_id = klifs_residues.Residues.apply(lambda y: y[3:])
klifs_residue_name + '   ' + klifs_residue_id

0       LYS   716
1       LYS   716
2       LYS   716
3       LYS   716
4       LYS   716
5       LYS   716
6       LYS   716
7       LYS   716
8       LYS   716
9       LYS   716
10      LYS   716
11      LYS   716
12      LYS   716
13      LYS   716
14      LYS   716
15      LYS   716
16      LYS   716
17      LYS   716
18      LYS   716
19      LYS   716
20      LYS   716
21      LYS   716
22      VAL   717
23      VAL   717
24      VAL   717
25      VAL   717
26      VAL   717
27      VAL   717
28      VAL   717
29      VAL   717
30      VAL   717
31      VAL   717
32      VAL   717
33      VAL   717
34      VAL   717
35      VAL   717
36      VAL   717
37      VAL   717
38      LEU   718
39      LEU   718
40      LEU   718
41      LEU   718
42      LEU   718
43      LEU   718
44      LEU   718
45      LEU   718
46      LEU   718
47      LEU   718
48      LEU   718
49      LEU   718
50      LEU   718
51      LEU   718
52      LEU   718
53      LEU   718
54      LEU   718
55      LE

## Comparison of DoGSiteScorer and KLIFS pocket

Here we compare the above two results by calculating the set intersection between the residue IDs obtained from DoGSiteScorer and KLIFS and then calculating the percentage of coverage over the KLIFS residue IDs.  

In [26]:
# Get intersection
set_intersection = len(set(klifs_residue_id) and set(dss_residue_number))
set_intersection

62

In [27]:
# Percentage of DoGSiteScorer Coverage when compared with KLIFS database
percentage_of_dss_coverage = set_intersection / len(klifs_residue_id) * 100
percentage_of_dss_coverage

4.545454545454546

####  Get DSS coverage with KLIFS results

In [28]:
def DSS_coverage():
    
    """
    DSS_coverage - This function returns the percentage of coverage when the output from DoGSiteScorer 
    website is comapred with the KLIFS database residues
    
    Parameters
    ----------
    set_intersection: int
        Conatins the number of similar residue IDs in both the cases
        
    set_union: int
        Contains total number of residues in both the cases 
    
    Returns
    -------
    float
        Returns the percenatge of similarity between the results of DoGSiteScorer and KLIFS database     
    """
    set_intersection = len(set(klifs_residue_id) and set(dss_residue_number))
    set_union = len(set(klifs_residue_id) or set(dss_residue_number))
    percentage = set_intersection / set_union * 100
    return(percentage)

In [29]:
# Final output percentage
final_percent = DSS_coverage()
final_percent

72.94117647058823